In [206]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from pandas import DataFrame, read_csv
from pathlib import Path

In [207]:
news_df = read_csv(Path('../arquivos/BBAS3_cls.csv'), parse_dates=[4])

In [208]:
news_df.head()

,code,source,url,title,datetime,raw_text,prep_text,cls_influence
0,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/ceo-do-b...,CEO do Banco do Brasil diz que houve falha de ...,2021-02-12 10:41:35+00:00,André Brandão (Foto: Marcelo Camargo/ Agência ...,andré brand fot marcel camarg agênc brasilreut...,-1
1,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/despesas...,Despesas permanecerão sob rigoroso controle ne...,2021-02-12 07:10:00+00:00,André Brandão (Foto: Marcelo Camargo/ Agência ...,andré brand fot marcel camarg agênc brasil pre...,-1
2,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/bradesco...,"Bradesco, Itaú, Santander ou Banco do Brasil: ...",2021-02-12 17:41:41+00:00,SÃO PAULO – A temporada de resultados do quart...,são paul a tempor result quart trimestr 2020 a...,-1
3,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/andre-br...,Ações do Banco do Brasil fecham em queda de 5%...,2021-01-13 18:11:02+00:00,"SÃO PAULO – As ações do Banco do Brasil (, R$ ...",são paul as açõ banc brasil r 3755 -494 fech q...,-1
4,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/banco-do...,Banco do Brasil anuncia abertura de 14 novas a...,2021-02-03 13:55:00+00:00,Seguindo o planejamento de reestruturação orga...,segu planej reestrutur organizac anunci jan ba...,1


In [209]:
train, test, cls_train, cls_test = train_test_split(news_df['prep_text'], 
                                                    news_df['cls_influence'],
                                                    test_size = 0.1,
                                                    random_state = 42)

In [211]:
tfidf = TfidfVectorizer(ngram_range=(1,2)
                        , max_df=0.5
                        , sublinear_tf=True
                        , max_features=20000)

In [212]:
tfidf_train = tfidf.fit_transform(train)
tfidf_test = tfidf.transform(test)

In [220]:
df = DataFrame(tfidf_train.todense(), columns=tfidf.get_feature_names())
df.loc[2].nlargest(50)


atual capit               0.247524
capit vier                0.247524
condico tod               0.247524
participara               0.247524
condico                   0.194813
aco                       0.174248
vier                      0.150641
ordin compon              0.146192
ordinariascompon          0.146192
ordinariascompon atual    0.146192
orient negoci             0.146192
tod                       0.137180
sa solicit                0.136116
soc dobanc                0.136116
vier ser                  0.136116
capit soc                 0.133581
436                       0.128967
compon                    0.128967
emissa banc               0.128967
respost question          0.128967
ser distribu              0.128967
solicit inform            0.128967
aco emissa                0.123421
aco ordin                 0.123421
dobanc                    0.123421
dobanc brasil             0.123421
dobrasil sa               0.123421
homolog                   0.118891
age                 

In [214]:
print(test[:10])

84     são paul o banc brasil divulg result operac re...
260    são paul o banc brasil envi seguint comunic me...
45     são paul a equip invest hsbc revis nest quinta...
176    são paul a spinell divulg cart recomend seman ...
143    bloomberg — a sociedad ub group banc brasil ba...
125    bloomberg — o ub bb investment bank joint vent...
227    são paul o exercíci opç sobr açõ moviment nest...
118    são paul a brad corre promov cinc alter cart t...
9      são paul em destaqu sess dest quinta-f invest ...
90     são paul destaqu nest quinta-f 13 açõ indústr ...
Name: prep_text, dtype: object


In [215]:
model = MultinomialNB()
model.fit(tfidf_train, cls_train)

MultinomialNB()

In [216]:
model.predict(tfidf_test)

array([ 1, -1,  1,  1,  1,  1,  1,  1, -1, -1,  1, -1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1], dtype=int64)

In [217]:
train_score = model.score(tfidf_train, cls_train)
test_score = model.score(tfidf_test, cls_test)
print('Train accuracy: ', train_score)
print('Test accuracy: ', test_score)

Train accuracy:  0.9239543726235742
Test accuracy:  0.5666666666666667


In [218]:
news_df['cls_influence'][238]

1